In [1]:
import requests
from io import StringIO
import pandas as pd


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',  None)
#一開始就要設定



# 指定爬取月報的網址
url = 'https://mops.twse.com.tw/nas/t21/sii/t21sc03_106_1_0.html'
# 抓取網頁
r = requests.get(url)
r.encoding = 'big5'

dfs = pd.read_html(StringIO(r.text))

#print(dfs[0:3])

data = []
#print(dfs[50].shape[1])

#for i in range(len(dfs)):
    #print(dfs[i].shape)
#了解資料長相

for i in dfs:
    if i.shape[1] <= 11 and i.shape[1] > 5:
        data.append(i)        
df = pd.concat(data)
#從for開始到df等同以下
# 將dfs中，row長度介於5~11的table合併（這些才是我們需要的table，其他table不需要）
#df = pd.concat([df for df in dfs if df.shape[1] <= 11 and df.shape[1] > 5])

#將資料資料篩選
# 設定column名稱

df.columns = df.columns.get_level_values(1)

#print(df)
#print('__________分隔線__________')
# 將 df 中的當月營收用 .to_numeric 變成數字，再把其中不能變成數字的部分以 NaN 取代（errors='coerce'）
df['當月營收'] = pd.to_numeric(df['當月營收'], 'coerce')

#print(df)
#print('__________分隔線__________')
# 再把當月營收中，出現 NaN 的 row 用 .dropna 整行刪除
df = df[~df['當月營收'].isnull()]
#print(df)
#print('__________分隔線__________')
# 刪除「公司代號」中出現「合計」的行數，其中「～」是否定的意思
df = df[df['公司代號'] != '合計']
#print(df)
#print('__________分隔線__________')


# 將「公司代號」與「公司名稱」共同列為 df 的 indexes
df = df.set_index(['公司代號', '公司名稱'])
#print(df.shape)


df



,,備註,上月比較增減(%),上月營收,去年同月增減(%),去年當月營收,當月營收,前期比較增減(%),去年累計營收,當月累計營收
公司代號,公司名稱,,,,,,,,,
1101,台泥,-,-34.89,9418581,-6.75,6575590,6131670,-6.75,6575590,6131670
1102,亞泥,-,-30.90,6023954,-17.84,5066062,4162130,-17.84,5066062,4162130
1103,嘉泥,-,-39.25,382981,17.90,197328,232657,17.90,197328,232657
1104,環泥,-,-21.55,463842,-2.53,373311,363865,-2.53,373311,363865
1108,幸福,-,-8.33,345573,-9.04,348266,316773,-9.04,348266,316773
1109,信大,-,-35.25,514449,-1.68,338758,333060,-1.68,338758,333060
1110,東泥,-,-26.97,142422,-25.89,140339,104003,-25.89,140339,104003
1201,味全,-,-1.08,1226869,0.03,1213101,1213521,0.03,1213101,1213521
1203,味王,-,-3.83,565174,-1.67,552771,543487,-1.67,552771,543487


存檔csv (全版本通用)

In [2]:
# 把 df 存成 csv 檔，並且命名為「test.csv」，指定用「utf_8_sig」編碼
df.to_csv('每月營收.csv', encoding='utf_8_sig')

# 讀取名為「test.csv」的 csv 檔，並且指定其中欄位名稱為「公司代號」與「公司名稱」作為 df 的 indexes
df = pd.read_csv('每月營收.csv', index_col=['公司代號','公司名稱'])
df.head()

,,備註,上月比較增減(%),上月營收,去年同月增減(%),去年當月營收,當月營收,前期比較增減(%),去年累計營收,當月累計營收
公司代號,公司名稱,,,,,,,,,
1101,台泥,-,-34.89,9418581,-6.75,6575590,6131670,-6.75,6575590,6131670
1102,亞泥,-,-30.90,6023954,-17.84,5066062,4162130,-17.84,5066062,4162130
1103,嘉泥,-,-39.25,382981,17.90,197328,232657,17.90,197328,232657
1104,環泥,-,-21.55,463842,-2.53,373311,363865,-2.53,373311,363865
1108,幸福,-,-8.33,345573,-9.04,348266,316773,-9.04,348266,316773


存檔sqlite3 (全版本通用)

In [3]:
# --------------- #
# 存取 sqlite3 檔  #
# --------------- #

import sqlite3

# 把 df 存成名為「monthly_report」的 sqlite3 檔，其中 conn 是與 database 的連結
conn = sqlite3.connect('SQ_每月營收的sqlite檔案.sqlite3')
df.to_sql('每月營收2', conn, if_exists='replace')

# 讀取 sqlite3 中名為「monthly_report」的 table，並且指定其中欄位名稱為「公司代號」與「公司代號」作為 df 的 indexes
df = pd.read_sql('select * from 每月營收2', conn, index_col=['公司代號','公司名稱'])
df.head()

,,備註,上月比較增減(%),上月營收,去年同月增減(%),去年當月營收,當月營收,前期比較增減(%),去年累計營收,當月累計營收
公司代號,公司名稱,,,,,,,,,
1101,台泥,-,-34.89,9418581,-6.75,6575590,6131670,-6.75,6575590,6131670
1102,亞泥,-,-30.90,6023954,-17.84,5066062,4162130,-17.84,5066062,4162130
1103,嘉泥,-,-39.25,382981,17.90,197328,232657,17.90,197328,232657
1104,環泥,-,-21.55,463842,-2.53,373311,363865,-2.53,373311,363865
1108,幸福,-,-8.33,345573,-9.04,348266,316773,-9.04,348266,316773
